In [3]:
import sys
from pathlib import Path
import numpy as np

PROJECT_ROOT = Path("..").resolve()
sys.path.append(str(PROJECT_ROOT))

from src.model_gru import build_gru

data_path = PROJECT_ROOT / "runs" / "dataset.npz"
print("loading:", data_path)

data = np.load(data_path)
print("keys:", data.files)

X_train = data["X_train"]
y_train = data["y_train"]
X_val   = data["X_val"]
y_val   = data["y_val"]
X_test  = data["X_test"]
y_test  = data["y_test"]

input_shape = X_train.shape[1:]
print("input_shape:", input_shape)


loading: /root/household-power-forecasting/runs/dataset.npz
keys: ['X_train', 'y_train', 'X_val', 'y_val', 'X_test', 'y_test']
input_shape: (60, 1)


In [4]:
import tensorflow as tf

EPOCHS = 30
BATCH_SIZE = 64

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    )
]


In [ ]:
EPOCHS = 3
BATCH_SIZE = 2048  

gru = build_gru(
    input_shape=input_shape,
    units=16,       
    dropout=0.1,     
    lr=3e-3          
)

callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor="val_loss",
        patience=1,                
        restore_best_weights=True
    )
]

hist_gru = gru.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=callbacks,
    verbose=1
)

pred_gru = gru.predict(X_test, batch_size=BATCH_SIZE).reshape(-1)


I0000 00:00:1765803357.137208   69186 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38435 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:0f:00.0, compute capability: 8.0


Epoch 1/30


2025-12-15 12:56:00.026446: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


22698/22698 ━━━━━━━━━━━━━━━━━━━━ 97s 4ms/step - loss: 6.7834e-04 - mae: 0.0117 - val_loss: 4.8626e-04 - val_mae: 0.0081
Epoch 2/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 94s 4ms/step - loss: 6.4650e-04 - mae: 0.0113 - val_loss: 4.8686e-04 - val_mae: 0.0081
Epoch 3/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 95s 4ms/step - loss: 6.4137e-04 - mae: 0.0112 - val_loss: 4.9068e-04 - val_mae: 0.0089
Epoch 4/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 94s 4ms/step - loss: 6.2968e-04 - mae: 0.0111 - val_loss: 4.7970e-04 - val_mae: 0.0085
Epoch 5/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 94s 4ms/step - loss: 6.1735e-04 - mae: 0.0110 - val_loss: 4.7315e-04 - val_mae: 0.0082
Epoch 6/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 95s 4ms/step - loss: 6.1117e-04 - mae: 0.0110 - val_loss: 4.7182e-04 - val_mae: 0.0084
Epoch 7/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 94s 4ms/step - loss: 6.0767e-04 - mae: 0.0109 - val_loss: 4.8227e-04 - val_mae: 0.0091
Epoch 8/30
22698/22698 ━━━━━━━━━━━━━━━━━━━━ 117s 5ms/step - loss: 6.0219e-04 - mae: 0.0109 - v

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, pred_gru)
rmse = np.sqrt(mean_squared_error(y_test, pred_gru))

print("GRU  MAE :", mae)
print("GRU RMSE :", rmse)
